# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

In [1]:
# # TODO: 
# + language model 
# + bases guess off 2 preceeding words 
# + generate 3-5 texts 
# + calculate perplexity (use 10-50 stored sentences (store at the very beginning))

In [2]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

In [3]:
dvach = open('2ch_corpus.txt', encoding="utf-8").read()
news = open('lenta.txt', encoding="utf-8").read()

In [4]:
dvach[:1000]

' Анимублядский WebM-треддля приличных анимублядей и прочих аутистов. Безграмотное быдло с дубляжом, войсовером, порнографией и котиками, советы мерзких мокрописечников, вниманиебляди всех видов и прочее непотребство отправляется в порнотред <ссылка>.Для поиска сoуса видео сохраняем кадр (правый клик по видео) и ищем его на Для воспроизведения WebM с 10-битным цветом нужно установить плагин vlc ( ) и отключить встроенный в браузер плеер (media. webm. enabled=false в firefox).О кодировании WebMДоступные кодеки — VP8 и VP9 для видео, Vorbis и Opus для звука, максимальный размер файла — 10240КБ, всех файлов в посте — около 40МБ. Делать WebM можно научиться в вики треда: Там находится подробная информация о выборе и настройке кодеков на примерах использования консольных утилит ffmpeg, vpxenc и mkvmerge. Неочевидные моменты— libvorbis при указании битрейта (-b:a) работает в режиме CBR (постоянный битрейт), и это портит качество звука; для режима VBR вместо битрейта надо указывать качество (

In [5]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [6]:
norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [7]:
from collections import Counter

In [8]:
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)

In [9]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})

In [10]:
def compute_joint_proba(text, word_probas):
    prob = 0
    for word in normalize(text):
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += (np.log(1/len(norm_dvach)))
    
    return np.exp(prob)

In [11]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikol\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [13]:
sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach[:5000000])]
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [14]:
len(sentences_dvach), len(sentences_news)

(71993, 32250)

## Set aside sentences for perplexity check

In [15]:
sens_for_perplexity_dvach = []
n = 20
for i in range(n):
    sens_for_perplexity_dvach.append(sentences_dvach[i])
    sentences_dvach.pop(i)

len(sens_for_perplexity_dvach), len(sentences_dvach)

(20, 71973)

In [16]:
sens_for_perplexity_news = []
n = 20
for i in range(n):
    sens_for_perplexity_news.append(sentences_news[i])
    sentences_news.pop(i)

len(sens_for_perplexity_news), len(sentences_news)

(20, 32230)

from nltk.util import ngrams as ngr

In [17]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence, 2))
    trigrams_dvach.update(ngrammer(sentence, 3))


unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence, 2))
    trigrams_news.update(ngrammer(sentence, 3))

In [18]:
bigrams_dvach["<start> <start>"]

71973

In [19]:
new_tri_list = []
for trigram, count in trigrams_dvach.items():
    if "<start> <start>" in trigram:
        new_tri_list.append(count)

In [20]:
sum(new_tri_list)

71973

In [21]:
def compute_joint_proba_markov_assumption(text, word_counts, bigram_counts):
    prob = 0
    for ngram in ngrammer(['<start>'] + ['<start>'] + normalize(phrase) + ['<end>'], 2):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)
    
    return np.exp(prob)

In [22]:
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [44]:
matrix_dvach = lil_matrix((len(bigrams_dvach), 
                         len(unigrams_dvach)))

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {bigram:i for i, bigram in enumerate(id2bigram_dvach)}

for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    matrix_dvach[bigram2id_dvach[word1 + ' ' + word2], word2id_dvach[word3]] =  (trigrams_dvach[ngram]/
                                                                                 (bigrams_dvach[word1 + ' ' + word2]))

In [45]:
matrix_news = lil_matrix((len(bigrams_news), 
                         len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigram_news)}

for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    matrix_news[bigram2id_news[word1 + ' ' + word2], word2id_news[word3]] =  (trigrams_news[ngram]/
                                                                                 (bigrams_news[word1 + ' ' + word2]))

In [46]:
def generate(matrix, id2word, word2id, n=100, start='<start> <start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        probabilities = matrix[current_idx].toarray()[0]
        
        probabilities = np.clip(probabilities, 0, None) 
        probabilities[np.isnan(probabilities)] = 0 
        probabilities[np.isinf(probabilities)] = 0 
        
        probabilities[probabilities == 0] = 1e-10 
        probabilities[probabilities < 1e-10] = 1e-10 
        probabilities = probabilities.astype(np.float64) 
        
        probabilities /= probabilities.sum()
        probabilities[-1] = 1 - probabilities[:-1].sum()
      
        #chosen = np.random.choice(matrix.shape[1], p=probabilities)
        chosen = np.random.default_rng().choice(matrix.shape[1], p=probabilities)
        # просто выбирать наиболее вероятное продолжение не получится
        # можете попробовать раскоментировать следующую строчку и посмотреть что получается
#         chosen = matrix[current_idx].argmax()
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id[start]
        current_idx = chosen
    
    return ' '.join(text)

In [47]:
text_dvach_list = [generate(matrix_dvach, id2bigram_dvach, bigram2id_dvach).replace('<end>', '\n') for i in range(5)]
print(f'Кол-во текстов по типу двач: {len(text_dvach_list)}\nПример текста двач:\n{text_dvach_list[0]}')

Кол-во текстов по типу двач: 5
Пример текста двач:
бы подтянул не было же история плагин vlc порнографией и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов порнографией и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов порнографией и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов порнографией и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов порнографией и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов порнографией и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов порнографией и и котиками котиками советы советы мерзких мерз

In [48]:
text_news_list = [generate(matrix_news, id2bigram_news, bigram2id_news).replace('<end>', '\n') for i in range(5)]
print(f'Кол-во текстов по типу новостей: {len(text_news_list)}\nПример текста новостей:\n{text_dvach_list[0]}')

Кол-во текстов по типу новостей: 5
Пример текста новостей:
бы подтянул не было же история плагин vlc порнографией и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов порнографией и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов порнографией и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов порнографией и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов порнографией и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов порнографией и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов порнографией и и котиками котиками советы советы мерз

In [49]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [50]:
def compute_join_proba_markov_assumption(text, bigram_counts, trigram_counts):
    prob = 0
    tokens = text
    for ngram in ngrammer(['<start>'] + ['<start>'] + tokens + ['<end>'], 3):
        word1, word2, word3 = ngram.split()
        bigram = word1 + ' ' + word2
        if bigram in bigram_counts and ngram in trigram_counts:
            prob += np.log(bigram_counts[bigram]/trigram_counts[ngram])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)

In [51]:
# perplexity 
len(sens_for_perplexity_dvach), len(sens_for_perplexity_news)

(20, 20)

In [52]:
ps_dvach = []
for sent in sens_for_perplexity_dvach:
    prob, N = compute_join_proba_markov_assumption(sent, bigrams_dvach, trigrams_dvach)
    if not N:
        continue
    ps_dvach.append(perplexity(prob, N))

In [53]:
np.mean(ps_dvach)

9039.260746209444

In [54]:
ps_news = []
for sent in sens_for_perplexity_news:
    prob, N = compute_join_proba_markov_assumption(sent, bigrams_news, trigrams_news)
    if not N:
        continue
    ps_news.append(perplexity(prob, N))

In [55]:
np.mean(ps_news)

45401.24909243016

## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [56]:
# сделаем класс чтобы хранить каждый из лучей
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score 

In [57]:
def generate_with_beam_search(matrix, id2word, word2id, n=100, max_beams=5, start='<start>'):
    # изначально у нас один луч с заданным началом (start по дефолту)
    initial_node = Beam(sequence=[start], score=np.log1p(0))
    beams = [initial_node]
    
    for i in range(n):
        # делаем n шагов генерации
        new_beams = []
        # на каждом шаге продолжаем каждый из имеющихся лучей
        for beam in beams:
            # лучи которые уже закончены не продолжаем (но и не удаляем)
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue
            
            # наша языковая модель предсказывает на основе предыдущего слова
            # достанем его из beam.sequence
            #last_id = word2id[beam.sequence[-1]]
            last_2_word = beam.sequence[-2:]
            last_id = [word2id[word] for word in last_2_word]
            
            # посмотрим вероятности продолжений для предыдущего слова
            probas = matrix[last_id].toarray()[0]
            
            # возьмем топ самых вероятных продолжений
            top_idxs = probas.argsort()[:-(max_beams+1):-1]
            for top_id in top_idxs:
                # иногда вероятности будут нулевые, такое не добавляем
                if not probas[top_id]:
                    break
                
                # создадим новый луч на основе текущего и варианта продолжения
                new_sequence = beam.sequence + [id2word[top_id]]
                # скор каждого луча это произведение вероятностей (или сумма логарифмов)
                new_score = beam.score + np.log1p(probas[top_id])
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        # отсортируем лучи по скору и возьмем только топ max_beams
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]
    
    # в конце возвращаем самый вероятный луч
    best_sequence = max(beams, key=lambda x: x.score).sequence

    
    return ' '.join(best_sequence)

Пустой промт:

In [58]:
print(generate_with_beam_search(matrix_dvach, id2word_dvach, word2id_dvach, start='<start>'))

<start> а и битрейта котиками максимальный советы доступные мерзких media мокрописечников vlc вниманиебляди 10-битным всех на видов и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов видов и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов видов и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов видов и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов видов и и котиками котиками советы советы мерзких мерзких мокрописечников мокрописечников вниманиебляди вниманиебляди всех всех видов видов и и котиками котиками


In [59]:
print(generate_with_beam_search(matrix_news, id2word_news, word2id_news, start='<start>'))

<start> по в октября напряжения преследовать значительного артиллерию достиг разбили нынешнего <end>


Промт "да":

In [60]:
print(generate_with_beam_search(matrix_dvach, id2word_dvach, word2id_dvach, start='да'))

да воспроизводятся воспроизводятся только только указании указании звука звука webm webm его его видео видео уса уса o o по с клик дубляжом правый войсовером кадр порнографией сохраняем и видео котиками уса советы o мерзких что мокрописечников параметром вниманиебляди для всех для видов для и для котиками для советы для мерзких для мокрописечников для вниманиебляди для всех для видов для и для котиками для советы для мерзких для мокрописечников для вниманиебляди для всех для видов для и для котиками для советы для мерзких для мокрописечников для вниманиебляди для всех для видов для и для котиками для советы для мерзких для мокрописечников


In [61]:
print(generate_with_beam_search(matrix_news, id2word_news, word2id_news, start='да'))

да чтобы чтобы объяснены объяснены время время времени времени отмечали отмечали военные военные в в напряжения напряжения значительного значительного достиг достиг нынешнего прошлого милиции пришли всемирную убедить сосредоточенный комплекса сми надеются <end>


В целом, генерация вышла сравнимой с генерацией из первого задания, 
и хотя модель все равно выдает рекурсивно повторяющиеся цепочки 
слов, они не такие длинные, как могут возникать при генерации 
моделью без лучей.